In [38]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

## NaCl and DNA Concentration 
This notebook creates a grid sampling of varying NaCl and DNA concentrations. 

In [39]:
silica_molar_mass = 60.08 #g/mol
silica_density = 2.2 #g/cm3
silica_particle_diameter = 26 #nm
DOPC_molar_mass = 785 #g/mol 
DOPC_SA_molecule = 0.6 #nm2/molecule

In [40]:
def calculate_total_silica_NP_SA(silica_mass):
    silica_moles = silica_mass/silica_molar_mass/1000
    silica_atoms = silica_moles*6.02e23
    silica_NP_volume = 4/3*(silica_particle_diameter/2)**3*np.pi #nm3
    silica_atoms_volume = silica_mass/(silica_density*1e-18) #nm3
    silica_NP_quantitity = silica_atoms_volume/silica_NP_volume
    silica_NP_SA = 4*np.pi*(silica_particle_diameter/2)**2
    total_silica_NP_SA = silica_NP_SA*silica_NP_quantitity #nm2 
    return total_silica_NP_SA

def calculate_number_silica_NPS(silica_mass):
    silica_moles = silica_mass/silica_molar_mass/1000
    silica_atoms = silica_moles*6.02e23
    silica_NP_volume = 4/3*(silica_particle_diameter/2)**3*np.pi #nm3
    silica_atoms_volume = silica_mass/(silica_density*1e-18) #nm3
    silica_NP_quantitity = silica_atoms_volume/silica_NP_volume
    return silica_NP_quantitity



In [41]:
def calculate_total_DOPC_SA(DOPC_mass):
    DOPC_moles = DOPC_mass/DOPC_molar_mass/1000
    DOPC_molecules = DOPC_moles*6.02e23
    total_DOPC_SA = DOPC_molecules*DOPC_SA_molecule
    total_DOPC_SA = total_DOPC_SA/2
    return total_DOPC_SA

def DOPC_SA_to_Mass(DOPC_SA):
    DOPC_SA = DOPC_SA*2
    DOPC_molecules = DOPC_SA/DOPC_SA_molecule
    DOPC_moles = DOPC_molecules/6.02e23
    DOPC_mass = DOPC_moles*DOPC_molar_mass*1000
    return DOPC_mass


## Mass of Silica 

In [42]:
silica_DOPC_stock_conc = 2 #mg/mL 
silica_DOPC_OA_stock_volume = 100 #uL 
mass_silica_sample = silica_DOPC_stock_conc*silica_DOPC_OA_stock_volume/1000
sample_volume = 200 #uL 

## NaCl

In [43]:
conc_NaCl = np.array([0, 10, 20, 30, 40]) #mM
stock_NaCl = 200 #mM

In [44]:
volume_NaCl_stock = conc_NaCl*sample_volume/stock_NaCl
volume_NaCl_stock = np.round(volume_NaCl_stock)

## DNA-Chol

In [45]:
DNA_strands_per_particle = np.array([20, 60, 100, 140])
DNA_stock_conc = 60 #uM 

In [46]:
number_silica_NP = calculate_number_silica_NPS(mass_silica_sample)
number_DNA_strands = number_silica_NP*DNA_strands_per_particle
DNA_moles = number_DNA_strands/6.02e23*10**6 #umoles
DNA_conc = DNA_moles/(sample_volume*10**-6) #uM
DNA_stock_volume = np.round(DNA_conc*sample_volume/DNA_stock_conc)

## Create Meshgrid

In [47]:
y = volume_NaCl_stock
z = DNA_stock_volume

In [48]:
yy, zz = np.meshgrid(y, z)
#xx = xx.flatten().reshape(-1,1)
yy = yy.flatten().reshape(-1,1)
zz = zz.flatten().reshape(-1,1)
volumes = np.hstack((zz, yy))
vol_silica = np.array([silica_DOPC_OA_stock_volume]*len(volumes)).reshape(-1,1)
volumes = np.hstack((vol_silica, volumes))


## DNA Bridge

In [49]:
DNA_Bridge = np.round(volumes[:,-2].reshape(-1,1)/2)
No_Bridge = np.array([0]*(len(volumes))).reshape(-1,1)
vol_p1 = np.hstack((volumes, DNA_Bridge))
vol_p2 = np.hstack((volumes, No_Bridge))
volumes = np.vstack((vol_p1, vol_p2))

## Add to Volumes

In [50]:
water = sample_volume - np.sum(volumes, axis =1)
volumes = np.hstack((water.reshape(-1,1), volumes))

In [51]:
df = pd.DataFrame(volumes, columns = ['Water','Silica_DOPC (2mg/mL)','DNA-Chol (60uM)','NaCl (200mM)','DNA-Bridge(60uM)'])

In [52]:
df

,Water,Silica_DOPC (2mg/mL),DNA-Chol (60uM),NaCl (200mM),DNA-Bridge(60uM)
0,93.0,100.0,5.0,0.0,2.0
1,83.0,100.0,5.0,10.0,2.0
2,73.0,100.0,5.0,20.0,2.0
3,63.0,100.0,5.0,30.0,2.0
4,53.0,100.0,5.0,40.0,2.0
5,76.0,100.0,16.0,0.0,8.0
6,66.0,100.0,16.0,10.0,8.0
7,56.0,100.0,16.0,20.0,8.0
8,46.0,100.0,16.0,30.0,8.0
9,36.0,100.0,16.0,40.0,8.0


In [15]:
df.sum(axis=0)

Water                   2125.0
Silica_DOPC (2mg/mL)    4000.0
DNA-Chol (60uM)          860.0
NaCl (200mM)             800.0
DNA-Bridge(60uM)         215.0
dtype: float64

In [ ]:
#df.to_csv('../../Volumes/250225/Experiment_40_Samples.csv', index=False)